In [1]:
import sqlite3
from datetime import datetime
from itertools import groupby

In [2]:
# Подключение к базе данных (файл будет создан, если не существует)
conn = sqlite3.connect('db/trading_bot.db')

# Создание курсора
cursor = conn.cursor()

# Создание таблицы сделок
cursor.execute('''
CREATE TABLE IF NOT EXISTS deals (
    id INTEGER PRIMARY KEY,
    algorithm_name TEXT NOT NULL,
    type INTEGER NOT NULL,
    instrument TEXT NOT NULL,
    datetime DATETIME DEFAULT CURRENT_TIMESTAMP,
    price REAL NOT NULL,
    commission REAL NOT NULL,
    total REAL NOT NULL
)
''')

# Создание индексов
cursor.execute('CREATE INDEX IF NOT EXISTS idx_instrument_datetime ON deals (algorithm_name, datetime)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_datetime ON deals (datetime)')

# Закрытие соединения
conn.close()

In [4]:
def add_deal(algorithm_name, type, instrument, price, commission, total):
    conn = sqlite3.connect('db/trading_bot.db')
    cursor = conn.cursor()
    cursor.execute('''
    INSERT INTO deals (algorithm_name, type, instrument, price, commission, total)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (algorithm_name, type, instrument, price, commission, total))
    conn.commit()
    conn.close()

In [3]:
# добавление записи
# add_deal(f"RNFT_v2_c4_s2.py", 2, 'RNFT', 162.6, 0.9, 162.6-0.9)

In [5]:
def get_daily_totals():
    conn = sqlite3.connect('db/trading_bot.db')
    cursor = conn.cursor()

    # Получаем суммы по дням и тип последней записи за день
    cursor.execute('''
    SELECT 
        strftime('%Y-%m-%d', datetime) AS day, 
        SUM(total) AS daily_total,
        (SELECT type FROM deals d2 WHERE strftime('%Y-%m-%d', d2.datetime) = strftime('%Y-%m-%d', datetime) ORDER BY id DESC LIMIT 1) AS last_type,
        (SELECT total FROM deals d3 WHERE strftime('%Y-%m-%d', d3.datetime) = strftime('%Y-%m-%d', datetime) ORDER BY id DESC LIMIT 1) AS last_total,
        algorithm_name,
        count(*) AS cnt
    FROM deals
    GROUP BY algorithm_name, day
    ''')

    results = cursor.fetchall()

    # Форматируем и выводим результаты
    for result in results:
        day, daily_total, last_type, last_total, algorithm_name, cnt = result
        if last_type == 1:
            adjusted_total = daily_total - last_total
            print(f"День: {day}, {algorithm_name}, "
                  f"Итоговая сумма с корректировкой: {round(adjusted_total, 2)}, "
                  f"Записей: {cnt}")
        else:
            print(f"День: {day}, {algorithm_name}, "
                  f"Итоговая сумма: {round(daily_total, 2)}, "
                  f"Записей: {cnt}")

    conn.close()

In [6]:
def get_hourly_totals(date=None):
    if date is None:
        date = datetime.now().strftime('%Y-%m-%d')  # Текущая дата в формате строки

    conn = sqlite3.connect('db/trading_bot.db')
    cursor = conn.cursor()

    # Получаем кумулятивные итоги по каждому algorithm_name за указанный день по часам
    cursor.execute('''
    SELECT 
        algorithm_name,
        strftime('%Y-%m-%d %H', datetime) AS hour,
        SUM(total) OVER (PARTITION BY algorithm_name ORDER BY datetime) AS cumulative_total,
        type,
        ROW_NUMBER() OVER (PARTITION BY algorithm_name, strftime('%Y-%m-%d %H', datetime) ORDER BY datetime DESC) AS rn
    FROM deals
    WHERE date(datetime) = ?
    ''', (date,))

    results = cursor.fetchall()

    # Фильтрация результатов для исключения последней записи каждого часа, если её type=1
    filtered_results = [result for result in results if not (result[3] == 1 and result[4] == 1)]

    # Группировка и вывод результатов
    for algorithm_name, group in groupby(filtered_results, key=lambda x: x[0]):
        print(f"Algorithm: {algorithm_name}")
        for hour, hour_group in groupby(group, key=lambda x: x[1]):
            cumulative_total = max([x[2] for x in hour_group])  # Максимальное кумулятивное total за час
            hour_string = hour.split(" ")[-1]
            hour_int = int(hour_string)
            print(f"  {hour_int+3} : {round(cumulative_total, 2)}")

    conn.close()

In [19]:
get_daily_totals()

День: 2024-03-12, RNFT_v2_c3_s2.py, Итоговая сумма: -6.21, Записей: 68
День: 2024-03-12, RNFT_v2_c4_s2.py, Итоговая сумма: -8.04, Записей: 76
День: 2024-03-12, RNFT_v2_c4_s3.py, Итоговая сумма: -3.18, Записей: 52


In [20]:
get_hourly_totals()  # Для текущего дня
# get_hourly_totals('2023-01-01')  # Для конкретной даты

Algorithm: RNFT_v2_c3_s2.py
  10 : -0.56
  11 : 0.34
  12 : 0.46
  13 : 0.8
  14 : -1.82
  15 : -3.18
  16 : -3.9
  17 : -4.49
  18 : -6.21
Algorithm: RNFT_v2_c4_s2.py
  10 : -0.76
  11 : -162.26
  12 : -162.58
  13 : -1.99
  14 : -2.92
  15 : -4.53
  16 : -5.67
  17 : -6.02
  18 : -7.26
Algorithm: RNFT_v2_c4_s3.py
  11 : 1.07
  12 : 1.21
  13 : 1.64
  14 : -0.4
  15 : -1.23
  16 : -1.64
  17 : -1.64
  18 : -3.18
